In [27]:
import numpy as np
import pandas as pd
import yfinance as yf
from matplotlib import pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from keras.models import load_model
import plotly.graph_objs as go
ticker = "RELIANCE.BO"
end_date = pd.Timestamp.now(tz="Asia/Kolkata").replace(hour=0, minute=0, second=0, microsecond=0)  # Set time to midnight to include today's data
start_date = end_date - pd.DateOffset(days=2)
realdata = yf.download(ticker, start=start_date, end=end_date, interval="1m")
print(realdata)

[*********************100%%**********************]  1 of 1 completed
                                  Open         High          Low        Close  \
Datetime                                                                        
2023-10-27 09:15:00+05:30  2241.649902  2244.399902  2238.899902  2244.399902   
2023-10-27 09:16:00+05:30  2244.899902  2245.949951  2241.899902  2241.899902   
2023-10-27 09:17:00+05:30  2241.899902  2242.300049  2238.449951  2239.850098   
2023-10-27 09:18:00+05:30  2239.600098  2240.100098  2238.399902  2238.600098   
2023-10-27 09:19:00+05:30  2238.600098  2241.250000  2236.050049  2240.000000   
...                                ...          ...          ...          ...   
2023-10-27 15:25:00+05:30  2263.850098  2266.500000  2263.649902  2266.500000   
2023-10-27 15:26:00+05:30  2264.699951  2266.850098  2264.699951  2265.600098   
2023-10-27 15:27:00+05:30  2265.600098  2266.000000  2263.600098  2263.600098   
2023-10-27 15:28:00+05:30  2263.000000  

In [2]:
seven_data = pd.read_csv("Reliance_data_7days.csv")
seven_data

,Datetime,Open,High,Low,Close,Adj Close,Volume
0,2023-10-20 09:15:00+05:30,2301.000000,2302.050049,2297.449951,2299.649902,2299.649902,4993
1,2023-10-20 09:16:00+05:30,2299.600098,2301.399902,2299.050049,2299.050049,2299.050049,1025
2,2023-10-20 09:17:00+05:30,2299.149902,2303.399902,2299.149902,2303.300049,2303.300049,1176
3,2023-10-20 09:18:00+05:30,2302.699951,2305.000000,2302.000000,2305.000000,2305.000000,971
4,2023-10-20 09:19:00+05:30,2305.550049,2306.250000,2304.199951,2305.000000,2305.000000,869
...,...,...,...,...,...,...,...
1492,2023-10-26 15:25:00+05:30,2225.600098,2226.199951,2224.000000,2225.000000,2225.000000,1851
1493,2023-10-26 15:26:00+05:30,2224.750000,2225.500000,2223.649902,2224.000000,2224.000000,2220
1494,2023-10-26 15:27:00+05:30,2225.000000,2225.000000,2223.100098,2224.000000,2224.000000,2283
1495,2023-10-26 15:28:00+05:30,2225.000000,2225.000000,2222.050049,2224.000000,2224.000000,3520


In [3]:
realtimedata = realdata.Close

In [4]:
model = load_model('keras_model1.h5')

In [5]:
past_hund_data = seven_data.tail(100)
# past_hund_data

In [6]:
past_hund_data = past_hund_data.Close


In [7]:
past_hund_data

1397    2225.649902
1398    2225.399902
1399    2226.000000
1400    2224.899902
1401    2225.899902
           ...     
1492    2225.000000
1493    2224.000000
1494    2224.000000
1495    2224.000000
1496    2224.949951
Name: Close, Length: 100, dtype: float64

In [8]:
sc = MinMaxScaler(feature_range = (0, 1))

In [12]:
x_test=[]
y_test=[]
final_df = past_hund_data.append(realtimedata,ignore_index = True)
final_df

C:\Users\Sarath Babu\AppData\Local\Temp\ipykernel_13048\4260947181.py:3: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = past_hund_data.append(realtimedata,ignore_index = True)


0      2225.649902
1      2225.399902
2      2226.000000
3      2224.899902
4      2225.899902
          ...     
470    2266.500000
471    2265.600098
472    2263.600098
473    2263.000000
474    2265.250000
Name: Close, Length: 475, dtype: float64

In [16]:
final_df_2d = final_df.values.reshape(-1, 1)
final_df_2d.shape

(475, 1)

In [17]:
input_data =sc.fit_transform(final_df_2d)

In [18]:
x_test =[]
y_test=[]

for i in range(100,input_data.shape[0]):
  x_test.append(input_data[i-100:i])
  y_test.append(input_data[i,0])

In [19]:
x_test,y_test = np.array(x_test),np.array(y_test)
x_test.shape,y_test.shape

((375, 100, 1), (375,))

In [20]:
y_predicted = model.predict(x_test)

12/12 [==============================] - 2s 59ms/step


In [21]:
y_predicted.shape

(375, 1)

In [24]:
scaler = sc.scale_

In [25]:
scale_factor = 1/scaler[0]
y_predicted = y_predicted*scale_factor
y_test = y_test*scale_factor

In [26]:
plt.figure(figsize=(12,6))
plt.plot(y_test,'b',label = 'Original Price')
plt.plot(y_predicted,'r',label = 'Predicted Price')
plt.xlabel('Time')
plt.ylabel('Price')
plt.legend()
plt.show()

NameError: name 'plt' is not defined